In [1]:
%pwd

'/home/farel/deep_learning/chicken-desease-classification/ressearch'

In [2]:
import os

os.chdir("../")

%pwd

'/home/farel/deep_learning/chicken-desease-classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_filename: Path
    unzip_dir: Path

In [4]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_filename=config.local_data_filename,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [5]:
import os
from urllib import request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size


class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_filename):
            filename, headers = request.urlretrieve(url=self.config.source_url,
                                                    filename=self.config.local_data_filename)
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"{filename} already exits of size: {get_size(Path(self.config.local_data_filename))}")
            
            
    def extract_zipfile(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_filename, 'r') as zip_file:
            zip_file.extractall(unzip_path)

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e